In [1]:
import sys
import time
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')

import nltk
nltk.download('stopwords')

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
! git clone https://github.com/PANDASANG1231/Kaggle_FB3_ELL.git
! pip install catboost

Cloning into 'Kaggle_FB3_ELL'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 35 (delta 3), reused 32 (delta 3), pack-reused 0
Unpacking objects: 100% (35/35), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [3]:
import pandas as pd

data = pd.read_csv("/content/Kaggle_FB3_ELL/data/train.csv")
data_test = pd.read_csv("/content/Kaggle_FB3_ELL/data/test.csv")

In [4]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool

en_stopwords = stopwords.words("english")

data_train, data_valid = train_test_split(data, test_size=int(0.1*data.shape[0]), random_state=2022)
# data_train = data_train.sample(100)

data_train["text_processed"] = data_train["full_text"] \
    .apply(lambda x: x.lower()) \
    .apply(lambda x: re.sub(r'\W+', ' ', x)) \
    .apply(lambda row: " ".join([word for word in row.split(" ") if word not in en_stopwords]))

data_valid["text_processed"] = data_valid["full_text"] \
    .apply(lambda x: x.lower()) \
    .apply(lambda x: re.sub(r'\W+', ' ', x)) \
    .apply(lambda row: " ".join([word for word in row.split(" ") if word not in en_stopwords]))

data_test["text_processed"] = data_test["full_text"] \
    .apply(lambda x: x.lower()) \
    .apply(lambda x: re.sub(r'\W+', ' ', x)) \
    .apply(lambda row: " ".join([word for word in row.split(" ") if word not in en_stopwords]))


vectorizer = TfidfVectorizer(max_features=100)
# vectorizer = TfidfVectorizer(max_features=1000)
X_train = vectorizer.fit_transform(data_train['text_processed'])
X_valid = vectorizer.transform(data_valid['text_processed'])
X_test = vectorizer.transform(data_test['text_processed'])



y_train = data_train[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']].values
y_valid = data_valid[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']].values

train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

In [ ]:
# params_lgb = {
#     "n_estimators": 10,
#     "verbose": -1
# }

# model = MultiOutputRegressor(LGBMRegressor(**params_lgb))
# model.fit(train_s, y_train)

In [34]:
param = {'learning_rate': 0.01, 
          'depth': 4, 
          'l2_leaf_reg': 4, 
          'loss_function': 'MultiRMSE', 
          'eval_metric': 'MultiRMSE', 
          'task_type': 'CPU', 
          'iterations': 1,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian', 
          'allow_const_label': True, 
          # 'logging_level': 'Verbose',
          'logging_level': 'Silent',
          'random_state': 2022
         }

model2 = CatBoostRegressor(**param)

def mcrmse(y_pred, y_true):

    return (((y_pred - y_true) ** 2).mean(axis=0) ** 0.5).mean()

def cal_mcrmse(model, X, y):

    y_pred = model.predict(X)
    return mcrmse(y_pred, y)


for i in range(4000):

    if i == 0:
        model2.fit(train_pool, eval_set=valid_pool)
    else:
        model2.fit(train_pool, eval_set=valid_pool, init_model=model2)

    if i % 50 == 0:
        train_loss, valid_loss = cal_mcrmse(model2, X_train, y_train), cal_mcrmse(model2, X_valid, y_valid)
        print(f"Epoch: {i:04d}, Train loss: {train_loss:.6f}, Valid loss: {valid_loss:.6f}.")


Epoch: 0000, Train loss: 0.649147, Valid loss: 0.679935.
Epoch: 0050, Train loss: 0.631604, Valid loss: 0.660194.
Epoch: 0100, Train loss: 0.621031, Valid loss: 0.648495.
Epoch: 0150, Train loss: 0.613294, Valid loss: 0.640107.
Epoch: 0200, Train loss: 0.607140, Valid loss: 0.634116.
Epoch: 0250, Train loss: 0.602052, Valid loss: 0.629482.
Epoch: 0300, Train loss: 0.597834, Valid loss: 0.625607.
Epoch: 0350, Train loss: 0.594017, Valid loss: 0.622239.
Epoch: 0400, Train loss: 0.590589, Valid loss: 0.619695.
Epoch: 0450, Train loss: 0.587592, Valid loss: 0.617637.
Epoch: 0500, Train loss: 0.584814, Valid loss: 0.615445.
Epoch: 0550, Train loss: 0.582100, Valid loss: 0.613588.
Epoch: 0600, Train loss: 0.579672, Valid loss: 0.611694.
Epoch: 0650, Train loss: 0.577345, Valid loss: 0.609935.
Epoch: 0700, Train loss: 0.575122, Valid loss: 0.608539.
Epoch: 0750, Train loss: 0.573036, Valid loss: 0.607177.
Epoch: 0800, Train loss: 0.571086, Valid loss: 0.606291.
Epoch: 0850, Train loss: 0.5689

In [35]:
model2.save_model("/content/Kaggle_FB3_ELL/rst/models/baseline_model_cat.pkl")


In [36]:
model2 = CatBoostRegressor()

model2.load_model("/content/Kaggle_FB3_ELL/rst/models/baseline_model_cat.pkl")


df_rst = pd.DataFrame(
    model2.predict(X_test),
    columns=['cohesion','syntax','vocabulary',
             'phraseology','grammar','conventions']
)

data_submit = pd.concat([data_test[["text_id"]], df_rst], axis=1)
data_submit.to_csv("/content/Kaggle_FB3_ELL/data/submission.csv", index=False)